# Homework 7

Rowan Data Mining 2 - Ron Neely  
Due 11/29/18.  

Easy but painful.  But a good exercise in R or your tool of choice.  You can work with someone else.  

Download the Chronic Kidney Disease file from [uci](http://archive.ics.uci.edu/ml/machine-learning-databases/00336).  Use 7zip to unzip.
Description of data is [here](https://archive.ics.uci.edu/ml/datasets/Chronic_Kidney_Disease).
It needs a fair amount of cleanup which will give you some practice.  


## Part 1: Load and Clean Data

a. Open in text editor and remove all the intro nonsense (save it somewhere though so you know what the column headings etc. 
should be)  
b. Add column headings  
c. Make everything numeric.  E.g.  ckd=1, nonckd=0, Normal=0, Abnormal=1,  yes=1, no=0, etc.  
d. Change missing data (currently == ?) to average values for class.  For example if average BP of ckd is 100 then ? becomes 100, if it’s 80 for nonckd then assign to 80.  (for the 1 and 0 cases choose the majority value) for that class  
e. Write out file as a .csv file.

## Part 2: Predict ckd=1

f.	Run a bagging example and a boosting example and see which works best in predicting ckd=1  
g.	Save your new csv file and submit.  
h.	Note I gave this HW assignment a couple of years ago and almost no 2 files agreed.  I then cleaned it myself and it agreed with Matt Long.  I still don’t know if Matt and I were right or if someone else was more right.  It will be interesting to see what people come up with.

In [50]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore') # avoid showing seaborn and sklearn warnings
import collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io.arff import loadarff
import seaborn as sns
import sklearn
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, \
    recall_score, f1_score, roc_curve, roc_auc_score, auc
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
import sys
print(sys.version)

3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:27:44) [MSC v.1900 64 bit (AMD64)]


## Part 1: Load and Clean Data

a. Open in text editor and remove all the intro nonsense (save it somewhere though so you know what the column headings etc. 
should be)  


In [2]:
!pwd
!ls "../in"
!cp "../in/chronic_kidney_disease.arff" .

/e/r/rn/rw/dm2/hw/07/out
Chronic_Kidney_Disease.rar
chronic_kidney_disease.arff
chronic_kidney_disease.info.txt
chronic_kidney_disease_full.arff


Line number 399 in the original file had a missing 'yes,no' for one of the values.
Data starts on line 30 so this would be record 399-30 = 369.
The original line looked like this:

    75,70,1.020,0,0,normal,normal,notpresent,notpresent,107,48,0.8,144,3.5,13.6,46,10300,4.8,no,,no,no,good,no,no,notckd

Note the double comma for 'htn'.  Edited by hand and replace the ',,' with ','.

In [3]:
# vi chronic_kidney_disease.1.arff
!pwd
!ls

/e/r/rn/rw/dm2/hw/07/out
chronic_kidney_disease.1.arff
chronic_kidney_disease.arff
hw7_kidney_ron.ipynb


b. Add column headings  

Load the file with scipy.
Convert it to a pandas data frame with headings.

In [4]:
raw_data, meta = loadarff("chronic_kidney_disease.1.arff")
df = pd.DataFrame(data=raw_data, index=None, columns=meta)
print(df.shape)
print(df.dtypes)
df.head(3)

(400, 25)
Chronic_Kidney_Disease
age      float64
bp       float64
sg        object
al        object
su        object
rbc       object
pc        object
pcc       object
ba        object
bgr      float64
bu       float64
sc       float64
sod      float64
pot      float64
hemo     float64
pcv      float64
wbcc     float64
rbcc     float64
htn       object
dm        object
cad       object
appet     object
pe        object
ane       object
class     object
dtype: object


Chronic_Kidney_Disease,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,b'1.020',b'1',b'0',b'?',b'normal',b'notpresent',b'notpresent',121.0,...,44.0,7800.0,5.2,b'yes',b'yes',b'no',b'good',b'no',b'no',b'ckd'
1,7.0,50.0,b'1.020',b'4',b'0',b'?',b'normal',b'notpresent',b'notpresent',NaN,...,38.0,6000.0,NaN,b'no',b'no',b'no',b'good',b'no',b'no',b'ckd'
2,62.0,80.0,b'1.010',b'2',b'3',b'normal',b'normal',b'notpresent',b'notpresent',423.0,...,31.0,7500.0,NaN,b'no',b'yes',b'no',b'poor',b'no',b'yes',b'ckd'


Whoa.  Alot of these fields have messy 'b' strings.
Let's decode them to clean them up.  
Should be able to leave float64's alone.


In [5]:
def clean_val(v):
    if isinstance(v, np.float64) or isinstance(v, float):
        return np.float64(v)
    elif isinstance(v, bytes):
        return v.decode('utf-8')
    else:
        return str(v)
    
df = df.applymap(clean_val)
df.head(3)

Chronic_Kidney_Disease,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,1.020,1,0,?,normal,notpresent,notpresent,121.0,...,44.0,7800.0,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4,0,?,normal,notpresent,notpresent,NaN,...,38.0,6000.0,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2,3,normal,normal,notpresent,notpresent,423.0,...,31.0,7500.0,NaN,no,yes,no,poor,no,yes,ckd


c. Make everything numeric.  E.g.  ckd=1, nonckd=0, Normal=0, Abnormal=1,  yes=1, no=0, etc.  
d. Change missing data (currently == ?) to average values for class.  For example if average BP of ckd is 100 then ? becomes 100, if it’s 80 for nonckd then assign to 80.  (for the 1 and 0 cases choose the majority value) for that class  

Do this column by column.

In [6]:
def nan2mean(ser):
    if ser.isnull().any():
        print("True")
        mean = ser.mean()
        print(mean)
        ser = ser.fillna(mean)        
    print(ser.isnull().any())
    return ser

df['age'] = nan2mean(df['age'])

True
51.48337595907928
False


In [7]:
df['bp'] = nan2mean(df['bp'])

True
76.46907216494846
False


In [8]:
def map_num_mean(ser):
    print(ser.value_counts())
    def map_val(v):
        if '?' == v:
            return np.nan
        else:
            return np.float64(v)
    nan_ser = ser.map(map_val)
    mean = nan_ser.mean()
    print(mean)
    mean_ser = nan_ser.fillna(mean)
    print(mean_ser.value_counts())
    return nan_ser.fillna(mean)
        
df['sg'] = map_num_mean(df['sg'])

1.020    106
1.010     84
1.025     81
1.015     75
?         47
1.005      7
Name: sg, dtype: int64
1.0174079320113256
1.020000    106
1.010000     84
1.025000     81
1.015000     75
1.017408     47
1.005000      7
Name: sg, dtype: int64


In [9]:
df['al'] = map_num_mean(df['al'])

0    199
?     46
1     44
2     43
3     43
4     24
5      1
Name: al, dtype: int64
1.0169491525423728
0.000000    199
1.016949     46
1.000000     44
3.000000     43
2.000000     43
4.000000     24
5.000000      1
Name: al, dtype: int64


In [10]:
df['su'] = map_num_mean(df['su'])

0    290
?     49
2     18
3     14
1     13
4     13
5      3
Name: su, dtype: int64
0.45014245014245013
0.000000    290
0.450142     49
2.000000     18
3.000000     14
1.000000     13
4.000000     13
5.000000      3
Name: su, dtype: int64


In [11]:
def map_abnorm(ser):
    print(ser.value_counts())
    most_common_val_string = ser.value_counts().sort_values(ascending=False).index[0]
    def map_val(v):
        if v == most_common_val_string or v == '?':
            return 0
        else:
            return 1
    ab_ser = ser.map(map_val)
    print(ab_ser.value_counts())
    return ab_ser

df['rbc'] = map_abnorm(df['rbc'])

normal      201
?           152
abnormal     47
Name: rbc, dtype: int64
0    353
1     47
Name: rbc, dtype: int64


In [12]:
df['pc'] = map_abnorm(df['pc'])

normal      259
abnormal     76
?            65
Name: pc, dtype: int64
0    324
1     76
Name: pc, dtype: int64


In [13]:
df['pcc'] = map_abnorm(df['pcc'])

notpresent    354
present        42
?               4
Name: pcc, dtype: int64
0    358
1     42
Name: pcc, dtype: int64


In [14]:
df['ba'] = map_abnorm(df['ba'])

notpresent    374
present        22
?               4
Name: ba, dtype: int64
0    378
1     22
Name: ba, dtype: int64


In [15]:
df['bgr'] = nan2mean(df['bgr'])

True
148.0365168539326
False


In [16]:
df['bu'] = nan2mean(df['bu'])

True
57.425721784776904
False


In [17]:
df['sc'] = nan2mean(df['sc'])

True
3.0724543080939934
False


In [18]:
df['sod'] = nan2mean(df['sod'])

True
137.52875399361022
False


In [19]:
df['pot'] = nan2mean(df['pot'])

True
4.627243589743592
False


In [20]:
df['hemo'] = nan2mean(df['hemo'])

True
12.526436781609195
False


In [21]:
df['pcv'] = nan2mean(df['pcv'])

True
38.88449848024316
False


In [22]:
df['wbcc'] = nan2mean(df['wbcc'])

True
8406.122448979591
False


In [23]:
df['rbcc'] = nan2mean(df['rbcc'])

True
4.707434944237919
False


In [24]:
df['htn'] = map_abnorm(df['htn'])

no     251
yes    147
?        2
Name: htn, dtype: int64
0    253
1    147
Name: htn, dtype: int64


In [25]:
df['dm'] = map_abnorm(df['dm'])

no     261
yes    137
?        2
Name: dm, dtype: int64
0    263
1    137
Name: dm, dtype: int64


In [26]:
df['cad'] = map_abnorm(df['cad'])

no     364
yes     34
?        2
Name: cad, dtype: int64
0    366
1     34
Name: cad, dtype: int64


In [27]:
df['appet'] = map_abnorm(df['appet'])

good    317
poor     82
?         1
Name: appet, dtype: int64
0    318
1     82
Name: appet, dtype: int64


In [28]:
df['pe'] = map_abnorm(df['pe'])

no     323
yes     76
?        1
Name: pe, dtype: int64
0    324
1     76
Name: pe, dtype: int64


In [29]:
df['ane'] = map_abnorm(df['ane'])

no     339
yes     60
?        1
Name: ane, dtype: int64
0    340
1     60
Name: ane, dtype: int64


In [30]:
df['class'] = map_abnorm(df['class'])

ckd       250
notckd    150
Name: class, dtype: int64
0    250
1    150
Name: class, dtype: int64


In [31]:
df.isnull().any()

Chronic_Kidney_Disease
age      False
bp       False
sg       False
al       False
su       False
rbc      False
pc       False
pcc      False
ba       False
bgr      False
bu       False
sc       False
sod      False
pot      False
hemo     False
pcv      False
wbcc     False
rbcc     False
htn      False
dm       False
cad      False
appet    False
pe       False
ane      False
class    False
dtype: bool

e. Write out file as a .csv file.

In [32]:
df.dtypes

Chronic_Kidney_Disease
age      float64
bp       float64
sg       float64
al       float64
su       float64
rbc        int64
pc         int64
pcc        int64
ba         int64
bgr      float64
bu       float64
sc       float64
sod      float64
pot      float64
hemo     float64
pcv      float64
wbcc     float64
rbcc     float64
htn        int64
dm         int64
cad        int64
appet      int64
pe         int64
ane        int64
class      int64
dtype: object

In [36]:
df.to_csv("chronic_kidney_disease.csv")
!ls

chronic_kidney_disease.1.arff
chronic_kidney_disease.arff
chronic_kidney_disease.csv
hw7_kidney_ron.ipynb


## Part 2: Predict ckd=1

f.	Run a bagging example and a boosting example and see which works best in predicting ckd=1  

Let's run a random forest for bagging.

In [52]:
test_size = 0.3
random_state = 4444
np.random.seed(random_state)
y = df['class']
X = df.drop('class', axis=1)
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=test_size, random_state=random_state)

test_size = 0.3
random_state = 4444
np.random.seed(random_state)

algorithms = (
    RandomForestClassifier,
    GradientBoostingClassifier)

algorithm_names = [a.__name__ for a in algorithms]
score_names = ['accuracy', 'precision', 'recall', 'f1']
def test_score(estimator, xtr, xtst, ytr, ytst):
    estimator.fit(xtr, ytr)
    yh = estimator.predict(xtst)
    acc = accuracy_score(ytst, yh)
    prc = precision_score(ytst, yh)
    rcl = recall_score(ytst, yh)
    f1= f1_score(ytst,yh)
    return acc, prc, rcl, f1

def test_scores():
    dscores = collections.defaultdict(list)
    for algorithm in algorithms:
        acc, prc, rcl, f1 = test_score(algorithm(), Xtrain, Xtest, ytrain, ytest)
        dscores['accuracy'].append(acc)
        dscores['precision'].append(prc)
        dscores['recall'].append(rcl)
        dscores['f1'].append(f1)
    return pd.DataFrame(dscores, columns = score_names, index = algorithm_names)

scores = test_scores()
scores

,accuracy,precision,recall,f1
RandomForestClassifier,0.983333,1.0,0.945946,0.972222
GradientBoostingClassifier,1.000000,1.0,1.000000,1.000000


Wow.  GradientBoosting outperformed Random Forestl

g.	Save your new csv file and submit.  

h.	Note I gave this HW assignment a couple of years ago and almost no 2 files agreed.  I then cleaned it myself and it agreed with Matt Long.  I still don’t know if Matt and I were right or if someone else was more right.  It will be interesting to see what people come up with.